In [ ]:
import pandas as pd
import numpy as np

import dateutil
import datetime
import time
from time import sleep

from bs4 import BeautifulSoup
import requests
import random
import re
import html5lib
import html
import pickle as pk
from fake_useragent import UserAgent
import json

In [ ]:
import selenium

from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from pyvirtualdisplay import Display

import os

In [ ]:
# timing function
def timefunc(f):
    def f_timer(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        print f.__name__, 'took', end - start, 'seconds'
        return result
    return f_timer

## Get Recent Url

In [1]:
!pwd

/Users/feiwang/Documents/python_code/scraping


In [ ]:
# Setting up chromedriver
chromedriver = '/Users/feiwang/Documents/python_code/scraping/chromedriver'
os.environ['webdriver.chrome.driver'] = chromedriver
#chromedriver
driver = webdriver.Chrome(chromedriver)

In [ ]:
# go straight to most-recent pages
petitions_url = 'https://www.change.org/petitions#most-recent/' # up to 5084

@timefunc
def get_petition_urls(base, start, stop):
    d = {}
    for i in range(start, stop+1):
        driver.get(base + str(i))
        petitions = driver.find_elements_by_xpath('//div[@class = "petition-list"]//ol//li[@class = "petition"]')
        for p in petitions:
            d[p.get_attribute('data-id')] = p.get_attribute('data-url')
    return d

@timefunc
def chunk_petition_urls(dic, base, first, last, chunk):
    for c in range(first, last+1, chunk):
        d = get_petition_urls(base, c, c+chunk)
        dic.update(d)
        filename = 'master' + str(c+chunk) + '.pkl'
        with open(filename, 'wb') as f:
            pk.dump(dic,f,-1)
    return dic

In [ ]:
master = {}
urls1000 = chunk_petition_urls(master, petitions_url, 1, 1000, 100)
pk.dump(master, open('master_recent_url.pickle', 'wb'))

## get victorial url

In [ ]:
chromedriver = '/Users/feiwang/Documents/python_code/Petition Prediction/chromedriver'
os.environ['webdriver.chrome.driver'] = chromedriver
#chromedriver
driver = webdriver.Chrome(chromedriver)

victories_url = 'https://www.change.org/victories#most-recent/' 

@timefunc
def get_victory_urls(base, start, stop):
    d = {}
    for i in range(start, stop+1):
        driver.get(base + str(i))
        sleep(random.randint(1,3))
        victories = driver.find_elements_by_xpath('//div[@class = "petition-list"]//ol//li[@class = "petition"]')
        for i, v in enumerate(victories):
            d[i] = v.get_attribute('data-url')
    return d

@timefunc
def chunk_victory_urls(dic, base, first, last, chunk):
    for c in range(first, last+1, chunk):
        d = get_victory_urls(base, c, c+chunk)
        dic.update(d)
        filename = 'victories' + str(c+chunk) + '.pkl'
        with open(filename, 'wb') as f:
            pk.dump(dic,f,-1)
    return dic

In [ ]:
victories = {}
victories = get_victory_urls(victories_url, 1, 200)
filename = 'victories_final.pkl'
with open(filename, 'wb') as f:
    pk.dump(victories,f,-1)

In [ ]:
def get_petition_data_recent(user_agent,url ):
    req = urllib2.Request(url, headers={'User-Agent' : user_agents})
    print url
    con = urllib2.urlopen( req )
    html = con.read()
    soup = BeautifulSoup(html,"html.parser")
    ###########################extract title
    titles=''
    titles = soup.findAll("div", { "class" : "txt-c" })
    title=[]
    for element in titles:
        if element.select('h1')!=[]:
             title=element.select('h1')           
    title=re.sub('(<.*?>)'," ",str(title[0]))
    print title 

    ########################extract text
    texts=""
    texts=soup.findAll("div", { "class" : "rte js-description-content" })
    print texts

    ########################extract the supporter number
    Nsupporter_fling=''
    Nsupporter_fling=soup.findAll("div", { "class" : "col-xs-4 type-s js-mobile-supporter-count" })

    ugly_Nsupporter=Nsupporter_fling[0].select('strong')[0]
    N_supporters=re.sub('(<.*?>)'," ",str(ugly_Nsupporter))                       
    print N_supporters

    one_petition_info={url:[title,texts, N_supporters] }

    return one_petition_info


In [ ]:
recent_petition_dict={}
ua = UserAgent()
user_agents=[ua.ie, ua.msie,
              ua['Internet Explorer'], ua.opera,
              ua.chrome,ua.google,
              ua['google chrome'],
              ua.firefox,ua.ff,
              ua.safari,ua.random]

with open('master_recent_url.pickle','rb') as f:
    url_chuck=pk.load(f)
#petition_chuck={}
for url in url_chuck.values():
    one_petition=get_petition_data_recent(user_agents,str(url))
    sleep(random.randint(1,10))
    #petition_chuck.update(one_petition)
    recent_petition_dict.update(one_petition)
#pk.dump(petition_chunck,open('recent_petition_chunk'+str(i)+'.pkl','wb'))
#pk.dump(recent_petition_dict,open('recent_petition.pkl','wb'))

In [ ]:
clean_recent_dict={}
remove_fish=lambda x: re.sub('(<.*?>)'," ",x)
for key in recent_petition_dict:
    clean_array=[]
    for entry in recent_petition_dict[key][1]:
        if entry.select('p'):
            clean_array.append(entry.select('p'))
        else:
            clean_array=['NA']
        texts=[]
        if len(clean_array):
            for element in clean_array:
                soup=BeautifulSoup(str(element)).text
                texts.append(re.sub('(<p>)'," ",str(soup)))
    clean_recent_dict[key]=[ recent_petition_dict[key][0],texts,recent_petition_dict[key][2]]
            
        